In [2]:
import pandas as pd
import json

In [7]:
def identificar_redundancias(df, caracteristica_a_identificar):
    agrupar_por = ['RUT-DV'] + caracteristica_a_identificar
    agrupado = df.groupby(by = agrupar_por).sum()
    duplicados = agrupado[agrupado.index.get_level_values(0).duplicated(keep = False)]
    return duplicados

def hacer_cambiador_de_caracteristica_redundante(df_con_duplicados, es_de_contrato):
    if es_de_contrato:
        diccionario = {rut[0]: '1' for rut in df_con_duplicados.index[::2]}
    
    else:
        diccionario = dict(df_con_duplicados.index[::2])

    return diccionario

def cambiar_redundancias(df, caracteristica_a_cambiar):
    caracteristica_a_cambiar = False
    if caracteristica_a_cambiar == 'CONTRATO':
        caracteristica_a_cambiar = True

    caracteristica_redundante = identificar_redundancias(df, [caracteristica_a_cambiar])
    cambiador_caracteristica = hacer_cambiador_de_caracteristica_redundante(caracteristica_redundante)
    print(f'Se cambiará la sgte. característica: {caracteristica_a_cambiar}:\n'
          f'{json.dumps(cambiador_caracteristica, indent = 1)}')
    
    for rut, caracteristica in cambiador_caracteristica.items():
        df.loc[rut, caracteristica_a_cambiar] = caracteristica
    
    return df

def formatear_df(df):
    df['NOMBRE'] = df['NOMBRE'].str.strip()
    df['RUT-DV'] = df['RUT-DV'].str.strip().str.upper()

    df = df.set_index('RUT-DV')

    return df

In [8]:
df_15076 = pd.read_excel('input\\TODOS_15076.xlsx', header = 2)
df_18834 = pd.read_excel('input\\TODOS_18834.xlsx', header = 2)
df_19664 = pd.read_excel('input\\TODOS_19664.xlsx', header = 2)

honorarios = pd.read_excel('input\\PERC AGOSTO.xlsx')

In [9]:
columnas = ['RUT-DV', 'NOMBRE', 'UNIDAD', 'CARGO', 'TOTAL HABER']

df_15076 = df_15076[columnas]
df_18834 = df_18834[columnas]
df_19664 = df_19664[columnas]

juntas_contrato = pd.concat([df_15076, df_18834, df_19664])
juntas_contrato['TIPO CONTRATA'] = 1

columnas = ['RUT-DV', 'NOMBRE', 'UNIDAD O SERVICIO DONDE SE DESEMPEÑA', 'CARGO', 'VALOR TOTAL O BRUTO']
honorarios = honorarios[columnas].rename(columns = {'UNIDAD O SERVICIO DONDE SE DESEMPEÑA': 'UNIDAD', 'VALOR TOTAL O BRUTO': 'TOTAL HABER'})
honorarios['TIPO CONTRATA'] = 2

# contrato_honorarios = pd.concat([juntas_contrato, honorarios])

In [11]:
juntas_contrato = formatear_df(juntas_contrato)
juntas_contrato = cambiar_redundancias(juntas_contrato, 'NOMBRE')
juntas_contrato = cambiar_redundancias(juntas_contrato, 'CARGO')

KeyError: 'RUT-DV'

In [123]:
suma_haberes_por_unidad = limpio.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'UNIDAD']).sum()
suma_haberes_por_unidad.to_excel('agrupado_por_cc.xlsx')

suma_haberes_por_persona = limpio.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO']).sum()
suma_haberes_por_persona.to_excel('agrupado_por_persona.xlsx')

In [217]:
suma_haber_por_persona = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'TIPO CONTRATA']).sum().reset_index()
suma_haber_por_persona.to_excel('suma_haberes_por_persona_con_honorarios.xlsx')

suma_haber_por_persona = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'TIPO CONTRATA', 'UNIDAD']).sum()
suma_haber_por_persona.to_excel('suma_haberes_por_cc_con_honorarios.xlsx')

In [218]:
personas_por_unidad = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'UNIDAD']).sum()
mask_personas_que_trabajan_en_mas_de_un_cc = personas_por_unidad.index.get_level_values(0).duplicated(keep = False)
unidades_totales_personas_que_trabajan_en_mas_de_un_cc = personas_por_unidad[mask_personas_que_trabajan_en_mas_de_un_cc].index.get_level_values(2)

In [219]:
unidades_que_quedaran = unidades_totales_personas_que_trabajan_en_mas_de_un_cc[::2]

ruts_a_cambiar_unidad = ['10282072-K', '10976333-0', '12470454-5', '12736759-0', '13507846-8',
       '15188820-8', '15413864-1', '15956991-8', '16093747-5', '16172330-4',
       '16653286-8', '17269121-8', '18018768-5', '18464580-7', '18953398-5',
       '22955669-K', '24591684-1', '6004926-2', '6012866-9', '7259175-5',
       '7877320-0', '8666237-K', '9356279-8', '9381699-4', '9381872-5',
       '9457009-3']

diccionario_cambio_unidades = dict(zip(ruts_a_cambiar_unidad, unidades_que_quedaran))

In [220]:
nueva_contrata = contrata_y_honorarios.copy()
nueva_contrata = nueva_contrata.set_index('RUT-DV')

for rut in diccionario_cambio_unidades:
    nueva_contrata.loc[rut, 'UNIDAD'] = diccionario_cambio_unidades[rut]

nueva_contrata = nueva_contrata.reset_index()

In [221]:
desglose_sin_cc_repetidos = nueva_contrata.groupby(by = ['RUT-DV', 'NOMBRE', 'UNIDAD', 'CARGO']).sum()
desglose_sin_cc_repetidos.to_excel('desglose_sin_cc_repetidos.xlsx')